In [10]:
#Stitiching together .txt output files when prediction is run on cropped images
import cv2
import numpy as np
import os
import shapely
import re

def parse_annotations(file_path):
    """Parse YOLOv8 annotations from a text file, handling empty files safely."""
    annotations = []
    count = 1
    filename = os.path.splitext(os.path.basename(file_path))[0]

    if os.stat(file_path).st_size == 0:  # Check if the file is empty
        print(f"Warning: {file_path} is empty. No annotations parsed.")
        return annotations  # Return empty list

    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            parts = line.strip().split()
            if not parts:  # Skip empty lines
                continue
            cls = filename + '_roi' + str(count)
            bbox = list(map(float, parts[1:]))  # YOLO format: class, freehand array
            annotations.append((cls, bbox))
            count += 1
    return annotations


def save_annotations(file_path, annotations, final_width, final_height):
    """Save YOLOv8 annotations to a text file with normalized bounding box coordinates rounded to 6 decimal places."""
    with open(file_path, 'w') as file:
        yolo_standard = 0
        for cls, bbox in annotations:
            # Normalize and round bbox
            print(bbox)
            normalized_bbox = [
                round(coord / final_width, 6) if i % 2 == 0 else round(coord / final_height, 6)
                for i, coord in enumerate(bbox)
            ]
            # Write the normalized bbox to the file
            file.write(f"{yolo_standard} {' '.join(map(str, normalized_bbox))}\n")

def transform_annotations(annotations, offset_x, offset_y, scale_x=1.0, scale_y=1.0):
    """
    Transform annotations for freehand shapes by applying offsets and scaling.
    Redundant points are removed, and all coordinates are rounded to integers.
    
    Args:
        annotations (list): List of tuples where each tuple contains a class label 
                            and an array of points in the format 
                            [x1, y1, x2, y2, ..., xn, yn].
        offset_x (float): Horizontal offset to apply.
        offset_y (float): Vertical offset to apply.
        scale_x (float): Horizontal scaling factor. Default is 1.0.
        scale_y (float): Vertical scaling factor. Default is 1.0.
    
    Returns:
        list: Transformed annotations with updated coordinates, without redundant points.
    """
    transformed = []
    
    for cls, points in annotations:
        # Apply transformation to each point in the array
        transformed_points = []
        unique_points = set()  # Keep track of unique (x, y) pairs
        for i in range(0, len(points), 2):  # Process in pairs (x, y)
            # Transform coordinates and round to nearest integer
            x = round(points[i] * scale_x + offset_x)
            y = round(points[i + 1] * scale_y + offset_y)
            # Add unique points only
            if (x, y) not in unique_points:
                unique_points.add((x, y))
                transformed_points.extend([x, y])  # Append transformed (x, y)
                
        transformed_points.extend([transformed_points[0], transformed_points[1]])
        
        # Append the transformed annotation
        transformed.append((cls, transformed_points))
    
    return transformed

from shapely.geometry import Polygon

def find_overlapping_rois(annotations, coordinates):
    """
    Find and print the names of ROIs that overlap.
    
    Args:
        annotations (list): List of tuples where each tuple contains a class label 
                            and an array of points in the format 
                            [x1, y1, x2, y2, ..., xn, yn].
    """
    # Create a dictionary to hold the polygons with their class labels
    polygons = {}
    
    # Convert annotation points to Shapely polygons
    for cls, points in annotations:
        # Create a Shapely Polygon from the points
        polygon = Polygon([(points[i], points[i + 1]) for i in range(0, len(points), 2)])
        polygons[cls] = polygon

    # Check for overlaps
    overlapping_rois = set()
    roi_names = list(polygons.keys())
    
    for i in range(len(roi_names)):
        for j in range(i + 1, len(roi_names)):
            roi1 = roi_names[i]
            roi2 = roi_names[j]
            # Check if the polygons overlap
            if polygons[roi1].overlaps(polygons[roi2]):
                overlapping_rois.add((roi1, roi2))
    def extract_number_from_crop(string):
        match = re.search(r'crop[_]?(\d+)', string)  # Finds "crop1" or "crop_1"
        return int(match.group(1)) if match else None  # Extract just the number
        
    def compare_crop_numbers(string1, string2):
        # Split the strings by underscore
        parts1 = extract_number_from_crop(string1)
        parts2 = extract_number_from_crop(string2)
        
        # Extract the second part (the crop number)
        crop_number1 = int(parts1)  # "2" from "crop_2_roi1"
        crop_number2 = int(parts2)  # "2" from "crop_2_roi76"
        
        # Compare the crop numbers
        return crop_number1 == crop_number2

    def get_overlap_coordinates(string1, string2, coordinates):
        
        # Split the strings by underscore
        parts1 = extract_number_from_crop(string1)
        parts2 = extract_number_from_crop(string2)
        
        # Extract the second part (the crop number)
        crop_number1 = int(parts1)  # "2" from "crop2_roi1"
        crop_number2 = int(parts2)  # "2" from "crop2_roi76"
        
        # Unpack the boxes
        x1_1, y1_1, x2_1, y2_1 = coordinates[crop_number1-1]
        x1_2, y1_2, x2_2, y2_2 = coordinates[crop_number2-1]
        
        # Compute the coordinates of the intersection box
        x1_intersection = max(x1_1, x1_2)
        y1_intersection = max(y1_1, y1_2)
        x2_intersection = min(x2_1, x2_2)
        y2_intersection = min(y2_1, y2_2)
        
        # If there is no intersection, return None
        if x1_intersection >= x2_intersection or y1_intersection >= y2_intersection:
            return None  # No overlap
        
        # Return the coordinates of the overlap
        return (x1_intersection, y1_intersection, x2_intersection, y2_intersection)

    def calculate_roi_differences(roi, x_min, y_min, x_max, y_max):
        """
        Calculate the differences between the bounding box of the ROI and the provided coordinates.
        
        Args:
            roi: List of coordinates representing the ROI as [x1, y1, x2, y2, x3, y3, ...]
            x_min, y_min: The minimum x and y values (e.g., the lower-left corner of a rectangle).
            x_max, y_max: The maximum x and y values (e.g., the upper-right corner of a rectangle).
            
        Returns:
            roi_x_min_diff, roi_y_min_diff, roi_x_max_diff, roi_y_max_diff: The differences
            between the ROI's bounding box and the provided bounding box.
        """
        # Calculate the min and max x and y values of the ROI (bounding box)
        roi_x_min = min(roi[::2])  # Min of x1, x2, x3, ...
        roi_y_min = min(roi[1::2])  # Min of y1, y2, y3, ...
        roi_x_max = max(roi[::2])  # Max of x1, x2, x3, ...
        roi_y_max = max(roi[1::2])  # Max of y1, y2, y3, ...
        
        # Calculate the differences
        roi_x_min_diff = roi_x_min - x_min
        roi_y_min_diff = roi_y_min - y_min
        roi_x_max_diff = x_max - roi_x_max
        roi_y_max_diff = y_max - roi_y_max
        
        return roi_x_min_diff, roi_y_min_diff, roi_x_max_diff, roi_y_max_diff
    
    to_delete = set()  # Use a set to avoid duplicates

    for roi_pair in overlapping_rois:
        if not compare_crop_numbers(roi_pair[0], roi_pair[1]):
            x_min, y_min, x_max, y_max = get_overlap_coordinates(roi_pair[0], roi_pair[1], coordinates)
            
            roi1_coordinates = next(points for cls, points in annotations if cls == roi_pair[0])
            roi2_coordinates = next(points for cls, points in annotations if cls == roi_pair[1])
    
            roi1_metrics = calculate_roi_differences(roi1_coordinates, x_min, y_min, x_max, y_max)
            roi2_metrics = calculate_roi_differences(roi2_coordinates, x_min, y_min, x_max, y_max)
            
            # Compare areas and keep track of the smaller one
            roi1_val = min(roi1_metrics)
            roi2_val = min(roi2_metrics)
    
            if roi1_val < 0:
                to_delete.add(roi_pair[1])  # Add to set instead of list
            elif roi2_val < 0:
                to_delete.add(roi_pair[0])  # Add to set instead of list
            else:
                polygon1 = polygons[roi_pair[0]]
                polygon2 = polygons[roi_pair[1]]
                
                # Calculate the areas of the polygons
                area1 = polygon1.area
                area2 = polygon2.area
                if area1 < area2:
                    smaller_pair = roi_pair[0]
                    smaller_area = area1
                else:
                    smaller_pair = roi_pair[1]
                    smaller_area = area2
                to_delete.add(smaller_pair)  # Add to set instead of list
    
    # Return as a list (convert set to list)
    return list(to_delete)

def delete_cells(annotations, to_delete):
    """
    Remove annotations from the list that match the ROI names in the to_delete list.
    
    Args:
        annotations (list): List of tuples, where each tuple contains a class label and an array of points.
        to_delete (list): List of ROI names to delete.
        
    Returns:
        list: A new list with the annotations that are not in the to_delete list.
    """
    # Filter annotations to exclude those whose ROI name is in the to_delete list
    updated_annotations = [annotation for annotation in annotations if annotation[0] not in to_delete]
    
    return updated_annotations
        

def merge_crops(images, annotations_files, coordinates, output_image_path, output_annotations_path):
    """Merge crops into one image and compile annotations."""
    # Initialize variables for stitching
    final_height = max(y2 for _, y1, _, y2 in coordinates)
    final_width = max(x2 for x1, _, x2, _ in coordinates)
    stitched_image = np.zeros((final_height, final_width, 3), dtype=np.uint8)

    # List to hold all annotations
    final_annotations = []

    # Iterate through each crop
    for i, (image_path, annotation_path, (x1, y1, x2, y2)) in enumerate(zip(images, annotations_files, coordinates)):
        # Read and place the image
        crop_image = cv2.imread(image_path)
        print((image_path, annotation_path, (x1, y1, x2, y2)))
        print(crop_image)
        stitched_image[y1:y2, x1:x2] = crop_image

        # Transform annotations
        crop_annotations = parse_annotations(annotation_path)
        transformed_annotations = transform_annotations(crop_annotations, x1, y1)
        #transformed_annotations = transform_annotations(crop_annotations, x1, y1, x2-x1, y2-y1)

        # Add transformed annotations to the final list
        final_annotations.extend(transformed_annotations)
        
    print(len(final_annotations))
    to_delete = find_overlapping_rois(final_annotations, coordinates)
    print(len(to_delete))
    final_annotations = delete_cells(final_annotations, to_delete)
    print(len(final_annotations))
    
    draw_rois_matplotlib(final_annotations, output_image_path, final_width, final_height)
    
    # Save final annotations
    save_annotations(output_annotations_path, final_annotations, final_width, final_height)

    # Save final image
    cv2.imwrite(output_image_path, stitched_image)

import cv2
import numpy as np
import matplotlib.pyplot as plt

def draw_rois_matplotlib(rois, image_path=None, final_width=None, final_height=None):
    """
    Overlay ROIs onto an image (provided as a file path) and display using Matplotlib.
    
    Args:
        rois (list): List of tuples where each tuple contains a class label 
                     and an array of points in the format 
                     [x1, y1, x2, y2, ..., xn, yn].
        image_path (str): Path to the background image to overlay ROIs.
                          If None, a blank image is created.
        final_width (int): Width of the image (required if `image_path` is None).
        final_height (int): Height of the image (required if `image_path` is None).
    """
    if image_path is not None:
        # Load the image from the provided file path
        image = cv2.imread(image_path)
        if image is None:
            raise FileNotFoundError(f"Image file not found at path: {image_path}")
        
        # Get the image dimensions
        final_height, final_width = image.shape[:2]
    else:
        if final_width is None or final_height is None:
            raise ValueError("final_width and final_height must be provided if no image is given.")
        # Create a blank white image
        image = np.ones((final_height, final_width, 3), dtype=np.uint8) * 255

    # Iterate through the ROIs
    for cls, points in rois:
        # Convert points into a format compatible with OpenCV
        polygon = np.array(points, dtype=np.int32).reshape(-1, 1, 2)
        
        # Draw the polygon on the image
        color = (255, 255, 0)  # Red color for the ROI
        thickness = 2
        cv2.polylines(image, [polygon], isClosed=True, color=color, thickness=thickness)
        
        # Optionally, draw the label near the ROI
        text_position = (polygon[0][0][0], polygon[0][0][1] - 10)  # Slightly above the first point
        cv2.putText(image, cls, text_position, cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1, cv2.LINE_AA)

    # Convert the image from BGR to RGB for Matplotlib
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Display the image using Matplotlib
    plt.figure(figsize=(10, 10))
    plt.imshow(image_rgb)
    plt.axis('off')  # Hide axes for better visualization
    plt.title("ROIs Visualization")
    plt.show()

# Example usage
if __name__ == "__main__":
    # Input data
    images = [r"path to crop image 1", r"path to crop image 2", 
]
    annotations_files = [r"path to .txt file for crop image 1", r"path to .txt file for crop image 2"
        
        #r"C:\Users\Mouse\Downloads\final_runs\ST\txts\DRG1\DRG1_crop1.txt", r"C:\Users\Mouse\Downloads\final_runs\ST\txts\DRG1\DRG1_crop2.txt", 
]
    coordinates = [
        (0, 0, 2400, 5797),     # First crop coordinates (same coordinates used to crop the full image)
        (2200, 0, 7261, 5797), # Second crop coordinates
    ]

    # Output paths
    output_image_path = r"path to original "
    output_annotations_path = r"C:\Users\Mouse\Downloads\YOLO_new_merged\DRG1\DRG1.txt"

    merge_crops(images, annotations_files, coordinates, output_image_path, output_annotations_path)
    
    images = [r"C:\Users\Mouse\Downloads\YOLO_new_merged\DRG1\DRG1\crop_1.tiff", r"C:\Users\Mouse\Downloads\YOLO_new_merged\DRG1\DRG1\crop_2.tiff"
       
    ]
    annotations_files = [r"C:\Users\Mouse\Downloads\YOLO_new_merged\DRG1\crop_1.txt", r"C:\Users\Mouse\Downloads\YOLO_new_merged\DRG1\crop_2.txt"
        
        #r"C:\Users\Mouse\Downloads\final_runs\ST\txts\DRG1\DRG1_crop1.txt", r"C:\Users\Mouse\Downloads\final_runs\ST\txts\DRG1\DRG1_crop2.txt"
    ]
    coordinates = [
        (0, 0, 2400, 5797),     # First crop
        (2200, 0, 7261, 5797), # Second crop
    ]
    output_image_path = r"C:\Users\Mouse\Downloads\YOLO_new_merged\DRG1\DRG1\Aligned_DRG1.tif"
    output_annotations_path = r"C:\Users\Mouse\Downloads\YOLO_new_merged\DRG1\DRG1.txt"
    # Merge crops and save results
    merge_crops(images, annotations_files, coordinates, output_image_path, output_annotations_path)

('C:\\Users\\Mouse\\Downloads\\YOLO_new_merged\\DRG1\\DRG1\\crop_1.tiff', 'C:\\Users\\Mouse\\Downloads\\YOLO_new_merged\\DRG1\\crop_1.txt', (0, 0, 2400, 5797))
[[[ 0  0 26]
  [ 0  0 29]
  [ 0  0 23]
  ...
  [ 0  0 22]
  [ 0  0 23]
  [ 0  0 24]]

 [[ 0  0 21]
  [ 0  0 26]
  [ 0  0 22]
  ...
  [ 0  0 23]
  [ 0  0 24]
  [ 0  0 29]]

 [[ 0  0 20]
  [ 0  0 22]
  [ 0  0 21]
  ...
  [ 0  0 25]
  [ 0  0 25]
  [ 0  0 24]]

 ...

 [[ 0  0  0]
  [ 0  0  0]
  [ 0  0  0]
  ...
  [ 0  0  0]
  [ 0  0  0]
  [ 0  0  0]]

 [[ 0  0  0]
  [ 0  0  0]
  [ 0  0  0]
  ...
  [ 0  0  0]
  [ 0  0  0]
  [ 0  0  0]]

 [[ 0  0  0]
  [ 0  0  0]
  [ 0  0  0]
  ...
  [ 0  0  0]
  [ 0  0  0]
  [ 0  0  0]]]
('C:\\Users\\Mouse\\Downloads\\YOLO_new_merged\\DRG1\\DRG1\\crop_2.tiff', 'C:\\Users\\Mouse\\Downloads\\YOLO_new_merged\\DRG1\\crop_2.txt', (2200, 0, 7261, 5797))
[[[ 0  0 18]
  [ 0  0 20]
  [ 0  0 21]
  ...
  [ 0  0  0]
  [ 0  0  0]
  [ 0  0  0]]

 [[ 0  0 27]
  [ 0  0 22]
  [ 0  0 22]
  ...
  [ 0  0  0]
  [ 0  0  0

ValueError: A linearring requires at least 4 coordinates.